In [1]:
#Caricare le librerie
import tweepy
import pandas as pd

In [2]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [3]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.

hashtags=["#lockdown","#novax","#NoGreenPass"]
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]
n=10000

for tweet in tweepy.Cursor(api.search_tweets,q=hashtags,count=n,lang="it").items(n):

  #gestione dei retweet con try-except
  try:
    list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
    list_rt_id.append(tweet.retweeted_status.user.id)

    #lista tweets che sono stati retweettati
    try: 
      rt_text=tweet.retweeted_status.extended_tweet["full_text"] 
    except AttributeError:
      rt_text=tweet.retweeted_status.text

    list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, rt_text, 
                     tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                     tweet.retweeted_status.user.screen_name]) 
  except:
    list_originals.append(0)
    list_rt_id.append(0) 

  try: 
    tweet_text=tweet.extended_tweet["full_text"]
  except AttributeError:
    tweet_text=tweet.text

  list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet_text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])


In [4]:
#creao df dei tweet
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name"])

tweets["id_retweet"]=list_originals[1:]
tweets["user_id_retweet"]=list_rt_id[1:]


#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted[1:], 
                      columns=['date','id','author_id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,id_retweet,user_id_retweet
0,2021-12-26 16:28:52+00:00,1475141575122501633,1301626694890139649,RT @AlexS8338: Non c'è mai stato alcun #lockdo...,0,1,📓TRIESTE,uomok58,True,None,1475100536093528082,1460634559113043973
1,2021-12-26 16:27:16+00:00,1475141169176883202,389380746,RT @NicolasVanHort1: Avete notato come sono sc...,0,5,"Firenze, Toscana",TuscanyWay,True,None,1475079249581252616,1417045292667576323
2,2021-12-26 16:25:41+00:00,1475140771032481796,240626745,RT @NicolasVanHort1: Avete notato come sono sc...,0,5,,MondoTlc,True,None,1475079249581252616,1417045292667576323
3,2021-12-26 16:05:45+00:00,1475135755034968068,86938862,Le #fakenews ormai sono all'ordine del giorno\...,0,0,,DemosLandia,False,None,0,0
4,2021-12-26 13:45:48+00:00,1475100536093528082,1460634559113043973,Non c'è mai stato alcun #lockdown dei non vacc...,5,1,"Milano, Lombardia",AlexS8338,False,None,0,0


In [5]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,author_id,text,like,n_rt,author
28,2021-12-18 19:27:13+00:00,1472287351640580103,493838022,I vaccini funzionano. I tamponi no.\n\nMa i va...,215,72,gabrygabr
1,2021-12-26 12:21:13+00:00,1475079249581252616,1417045292667576323,Avete notato come sono scoppiati i contagi neg...,26,5,NicolasVanHort1
52,2021-12-18 19:28:06+00:00,1472287574894989312,984126296094597121,#lockdown inevitabile senza #ObbligoVaccinale ...,35,3,cigolionocingue
42,2021-12-18 22:12:01+00:00,1472328828588634117,1009125592799875072,"@ilmessaggeroit Signore e signori, ecco a voi ...",5,1,LMaledetta
32,2021-12-18 22:35:18+00:00,1472334684461879299,1009125592799875072,Il #vaccino ha creato:\nUna FALSA copertura\nU...,4,1,LMaledetta
19,2021-12-18 22:16:54+00:00,1472330054533361670,39511932,"Se l’Italia farà un #lockdown per tutti, gran ...",26,1,LucaGalizia
17,2021-12-19 09:00:27+00:00,1472492008769998855,1010807990,Negazionista è chi ancora non vede la verità \...,1,1,PaperinikEdonna
15,2021-12-19 15:05:25+00:00,1472583856976973830,1010807990,307 arresti cardiaci con patologie gravi e 175...,0,1,PaperinikEdonna
11,2021-12-19 08:34:35+00:00,1472485501215121408,1397982227925975045,Io dico che starete buoni: sono 20 mesi che pr...,7,1,strozzabistex
7,2021-12-21 06:08:01+00:00,1473173393298141188,880502911,#novax #ottoemezzo #21dicembre #NoGreenPass #n...,1,1,rotre54


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [6]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                     id  like  n_rt           author  count()
0   1472287351640580103   215    72        gabrygabr       40
1   1475079249581252616    26     5  NicolasVanHort1        4
2   1472287574894989312    35     3  cigolionocingue        3
3   1475100536093528082     5     1        AlexS8338        1
4   1474086850319065090     3     1       LMaledetta        1
5   1473284205413113856     1     1        AlexS8338        1
6   1473173393298141188     1     1          rotre54        1
7   1472583856976973830     0     1  PaperinikEdonna        1
8   1472492008769998855     1     1  PaperinikEdonna        1
9   1472485501215121408     7     1    strozzabistex        1
10  1472334684461879299     4     1       LMaledetta        1
11  1472330054533361670    26     1      LucaGalizia        1
12  1472328828588634117     5     1       LMaledetta        1


#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [7]:
rt_tweets=tweets[tweets.retweet==True]

In [8]:
tweets_ids=rt_tweets[['user_id','author']]
retwitted_ids=retweetted[['author_id','author']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates().reset_index(drop=True)

In [9]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None})

In [10]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [11]:
Nodes_df.to_csv('Nodes_df1.csv', index=False)
Edges_df.to_csv('Edges_df1.csv', index=False)

##### Grafi con NetworkX

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
Edges_df=pd.read_csv("C:/Users/dvd_m/Desktop/Social Media Analytics/SMA-Project/Edges_df.csv")
G = nx.from_pandas_edgelist(Edges_df, source='Source', target='Target')
df_cluster=pd.DataFrame({"nodes":nx.clustering(G).keys(),"cluster":nx.clustering(G).values()})
grafo=pd.merge(Edges_df,df_cluster, left_on="Source",right_on="nodes")